In [1]:
import rdflib

In [2]:
from rdflib import Namespace

In [3]:
owlNS = Namespace("http://www.w3.org/2002/07/owl#")
owlClass = owlNS["Class"]
owlObjectProperty = owlNS["ObjectProperty"]
owlDatatypeProperty = owlNS["DatatypeProperty"]
rdfNS = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfProperty = rdfNS["Property"]
rdfType = rdfNS["type"]
rdfsNS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
rdfsSubClassOf = rdfsNS["subClassOf"]
rdfsDomain = rdfsNS["domain"]
rdfsRange = rdfsNS["range"]
xsdNS = Namespace("http://www.w3.org/2001/XMLSchema#")
xsdString = xsdNS["string"]

In [4]:
petNS = Namespace('http://www.modrisco.github.io/pet#')

In [6]:
objectClass = petNS['Object']
petClass = petNS['Pet']
tagClass = petNS['Tag']
userClass = petNS['User']
orderClass = petNS['Order']
categotyClass = petNS['Categoty']

In [10]:
name = petNS['name']
# isA = petNS['is_a']
categotyOfPet = petNS['categoty_of_pet']
subClassOf = petNS['sub_class_of']

In [11]:
classTriples = [
# class declarations:
(petNS['Object'], rdfType, owlClass),
(petNS['Pet'], rdfType, owlClass),
(petNS['Tag'], rdfType, owlClass),
(petNS['User'], rdfType, owlClass),
(petNS['Order'], rdfType, owlClass),
(petNS['Categoty'], rdfType, owlClass)
]

In [12]:
classHeirarchyTriples = [
# class heirarchy:
(petNS['Category'], rdfsSubClassOf, petNS['Object']),
(petNS['Order'], rdfsSubClassOf, petNS['Object']),
(petNS['User'], rdfsSubClassOf, petNS['Object']),
(petNS['Tag'], rdfsSubClassOf, petNS['Category']),
(petNS['Pet'], rdfsSubClassOf, petNS['Object'])
]

In [13]:
propertyTriples = [
# name property:
(petNS['name'], rdfType, owlDatatypeProperty),
(petNS['name'], rdfsDomain, petNS['Object']),
(petNS['name'], rdfsRange, xsdString),
# # isA property:
# (petNS['is_a'], rdfType, owlObjectProperty),
# (petNS['is_a'], rdfsDomain, petNS['Pet']),
# (petNS['is_a'], rdfsRange, filmNS['Performance']),
# categotyOfPet property:
(petNS['categoty_of_pet'], rdfType, owlObjectProperty),
(petNS['categoty_of_pet'], rdfsDomain, petNS['Pet']),
(petNS['categoty_of_pet'], rdfsRange, petNS['Category']),
# subClassOf property:
(petNS['sub_class_of'], rdfType, owlObjectProperty),
(petNS['sub_class_of'], rdfsDomain, petNS['Tag']),
(petNS['sub_class_of'], rdfsRange, petNS['Category']),
]

In [14]:
from rdflib import ConjunctiveGraph
graph = ConjunctiveGraph()

In [15]:
for triple in classTriples:
    graph.add(triple)
for triple in classHeirarchyTriples:
    graph.add(triple)
for triple in propertyTriples:
    graph.add(triple)

In [16]:
list(graph.subjects(rdfType, owlClass))

[rdflib.term.URIRef('http://www.modrisco.github.io/pet#Categoty'),
 rdflib.term.URIRef('http://www.modrisco.github.io/pet#Order'),
 rdflib.term.URIRef('http://www.modrisco.github.io/pet#Pet'),
 rdflib.term.URIRef('http://www.modrisco.github.io/pet#User'),
 rdflib.term.URIRef('http://www.modrisco.github.io/pet#Tag'),
 rdflib.term.URIRef('http://www.modrisco.github.io/pet#Object')]

In [17]:
list(graph.subjects(rdfType, owlObjectProperty))

[rdflib.term.URIRef('http://www.modrisco.github.io/pet#categoty_of_pet'),
 rdflib.term.URIRef('http://www.modrisco.github.io/pet#sub_class_of')]

In [18]:
def isSubClassOf(subClass, superClass, graph):
    if subClass == superClass: 
        return True
    for parentClass in graph.objects(subClass, rdfsSubClassOf):
        if isSubClassOf(parentClass, superClass, graph):
            return True
        else: 
            return False

In [19]:
from rdflib import Literal
petTriples = [
# pet:
(petNS['Brownie'], rdfType, petNS['Pet']),
(petNS['Brownie'], petNS['name'],Literal("Brownie", datatype=xsdString)),
# (petNS['Brownie'], petNS['starring'], filmNS['bl_performance']),
# (petNS['Brownie'], petNS['directedBy'], filmNS['ridley_scott']),
# performance:
# (filmNS['bl_performance'], rdfType, filmNS['Performance']),
# (filmNS['bl_performance'], filmNS['hasActor'], filmNS['harrison_ford']),
# (filmNS['bl_performance'], filmNS['hasRole'], filmNS['rick_deckard']),
# category:
# (filmNS['harrison_ford'], rdfType, filmNS['Actor']),
# (filmNS['harrison_ford'], filmNS['name'],Literal("Harrison Ford", datatype=xsdString)),
# tag:
(petNS['Apartment_Friendly'], rdfType, petNS['Tag']),
(petNS['Apartment_Friendly'], petNS['sub_class_of'], petNS['Category']),
]
for triple in petTriples: 
    graph.add(triple)

In [20]:
def findInstances(queryClass, graph, instances=None):
    if instances is None: 
        instances = set()
    for instance in graph.subjects(rdfType, queryClass): 
        instances.add(instance)
    for subClass in graph.subjects(rdfsSubClassOf, queryClass):
        findInstances(subClass, graph, instances)
    return instances

In [22]:
findInstances(petClass, graph)

{rdflib.term.URIRef('http://www.modrisco.github.io/pet#Brownie')}

In [23]:
findInstances(tagClass, graph)

{rdflib.term.URIRef('http://www.modrisco.github.io/pet#Apartment_Friendly')}

In [24]:
graph.serialize(destination="graph.ttl", format="turtle")